In [8]:
from rdflib import Namespace, Graph, URIRef, Literal, XSD
import json

from get_schema_predicate import return_predicate

In [9]:
def str_split_to_lst(string):
    return string.split(', ')

In [10]:
print(return_predicate("categories"))

('https://schema.org/category', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))


In [11]:
file_path = "/Users/heede/Library/CloudStorage/OneDrive-AalborgUniversitet/DVML P7/Data/yelp_academic_dataset_business.json"

In [12]:
schema = Namespace("https://schema.org/")

yelp_business = Namespace("https://www.yelp.com/biz/")
yelp_user = Namespace("https://www.yelp.com/user_details?userid=")
yelp_review_vc = Namespace("https://www.yelp.com/kg/review/votes_count/")

In [13]:
G = Graph()
G.bind(prefix="biz", namespace=yelp_business)

In [14]:
with open(file=file_path, mode="r") as file:
    for line in file:
        try:
            line = json.loads(line)

            json_key = list(line.keys())[0]
            subject = line[json_key]
            del line[json_key]
            #print(json_key, _subject)

            for _predicate, _object in line.items():
                if isinstance(_object, type(None)):
                    pass
                elif isinstance(_object, dict):# or _predicate in ["categories", "hours", "attributes"]:
                    for key_pred, value_obj in _object.items():
                        predicate, object_type = return_predicate(key_pred)
                        G.add(triple=(URIRef(yelp_business + subject), # Subject
                                      URIRef(predicate), # Predicate
                                      Literal(value_obj, datatype=object_type) # Object
                              ))
                elif _predicate == "categories":
                    obj_lst = str_split_to_lst(_object)
                    predicate, object_type = return_predicate(_predicate)
                    # print(predicate, object_type)
                    for obj in obj_lst:
                        # print(obj)
                        G.add(triple=(URIRef(yelp_business + subject), # Subject
                                  URIRef(predicate), # Predicate
                                  Literal(obj, datatype=object_type) # Object
                          ))
                else:
                    predicate, object_type = return_predicate(_predicate)
                    G.add(triple=(URIRef(yelp_business + subject), # Subject
                                  URIRef(predicate), # Predicate
                                  Literal(_object, datatype=object_type) # Object
                          ))
        except Exception as e:
            pass
            if not isinstance(_object, dict):
                print(e)
                # print(subject, _predicate, _object)
                # print(type(_object))

KeyboardInterrupt: 

In [119]:
string = "Automotive, Car Rental, Hotels & Travel, Truck Rental"
lst = string.split(', ')
lst

['Automotive', 'Car Rental', 'Hotels & Travel', 'Truck Rental']

In [81]:
G.serialize(destination="kg.ttl")

<Graph identifier=N1f9dc0b92cc74ea2aa16d03505b861ed (<class 'rdflib.graph.Graph'>)>

In [100]:
dct = dict()
with open(file=file_path, mode="r") as file:
    for line in file:
        line = json.loads(line)
        # print(line["attributes"])
        #
        if isinstance(line["attributes"], dict):
            dct |= line["attributes"]
dct

{'ByAppointmentOnly': 'False',
 'BusinessAcceptsCreditCards': 'True',
 'BikeParking': 'False',
 'RestaurantsPriceRange2': '1',
 'CoatCheck': 'False',
 'RestaurantsTakeOut': 'None',
 'RestaurantsDelivery': 'None',
 'Caters': 'True',
 'WiFi': "u'free'",
 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
 'WheelchairAccessible': 'True',
 'HappyHour': 'False',
 'OutdoorSeating': 'False',
 'HasTV': 'True',
 'RestaurantsReservations': 'False',
 'DogsAllowed': 'True',
 'Alcohol': "'none'",
 'GoodForKids': 'True',
 'RestaurantsAttire': "u'casual'",
 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False}",
 'RestaurantsTableService': 'True',
 'RestaurantsGoodForGroups': 'True',
 'DriveThru': 'True',
 'NoiseLevel': "u'average'",
 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'break